<a href="https://colab.research.google.com/github/saminjamshidi/Question_generation/blob/master/question_generation_GAT_Bart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install transformers
!pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=058625c62259d1fe1185ff8fd9e0efcdaade3a3b095295c3665bf53877babbfd
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [ ]:
import nltk
import torch
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
import spacy
import networkx as nx
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


In [ ]:
def Dataload():
  from google.colab import drive

  drive.mount('/content/drive')
  import json

  file_path = '/content/drive/MyDrive/dataset-jonh-afterpreprocess/fullDataset.json'

  # Open the file
  with open(file_path) as file:
      # Read the file contents
      json_data = file.read()

  # Split the JSON data into individual objects
  json_objects = json_data.strip().split('\n')
  data=[]
  # Process each JSON object separately
  index=0
  for json_object in json_objects:
      # Load the JSON data
      index+=1
      data.append(json.loads(json_object))
      if index == 100:
        break
  return data

In [ ]:
P=[]
def build_entity_graph(text):
    # Initialize Stanford CoreNLP parser
    # parser = CoreNLPParser(url='http://localhost:9000')

    # Extract named entities from the context
    nlp = spacy.load("en_core_web_sm")
    # doc = nlp(context)
    p_entities=[]
    entities = []

    for paragraph in text.values():
        P.append(paragraph['context'])
        # print("paragraph['context']",paragraph['context'])
        entities.extend(nlp(paragraph['context']).ents)
        # entities.extend(extract_entities(paragraph['context']))
        # print("entities",entities)
    N = len(entities)

    # Construct entity graph
    entity_graph = {
        'sentence_level_links': set(),
        'context_level_links': set(),
        'paragraph_level_links': set()
    }

    # Sentence-level links
    for paragraph in text.values():
        # print("Sentence-level links paragraph['context']",paragraph['context'])
        sentences = nltk.sent_tokenize(paragraph['context'])
        # print("Sentence-level links sentences",sentences)
        for sentence in sentences:
            sentence_entities = set(nlp(sentence).ents)
            for entity1 in sentence_entities:
                for entity2 in sentence_entities:
                    if entity1 != entity2:
                        entity_graph['sentence_level_links'].add((entity1, entity2))

    # Context-level links
    for paragraph in text.values():
        # print("paragraph['context']",paragraph['context'])
        p_entities.extend(nlp(paragraph['context']).ents)
        for entity1 in p_entities:
           for entity2 in p_entities:
            if entity1 != entity2:
                entity_graph['context_level_links'].add((entity1, entity2))

    # Paragraph-level links
    for p1 in text.values():
        p1_entities = set(nlp(p1['context']).ents)
        # print(p1_entities)
        for p2 in text.values():
            if p1 != p2:
                p2_entities = set(nlp(p2['context']).ents)
                # print(p2_entities)
                for entity1 in p1_entities:
                    for entity2 in p2_entities:
                        entity_graph['paragraph_level_links'].add((entity1, entity2))

    return entity_graph, N,entities


In [ ]:
from transformers import BertTokenizer
import networkx as nx  # Import the NetworkX library for creating the graph

# model_name = 'bert-base-uncased'
# tz = BertTokenizer.from_pretrained(model_name)

def generate_graph(entities, entity_graph):
    unique_values = {}
    unique_values_tensor={}
    edge = []
    nodes=[]
    index=0
    for i in entities:
        if i.text not in unique_values:

            unique_values[i.text]=len(unique_values)
            nodes.append([len(unique_values)])


    for i in entity_graph['sentence_level_links']:
        source_value = i[0].text
        target_value = i[1].text
        if source_value in unique_values and target_value in unique_values:
          if [unique_values[source_value], unique_values[target_value]] not in  edge:
            edge.append([unique_values[source_value], unique_values[target_value]])
        # if [unique_values[source_value], unique_values[target_value]]  not in edge:
        #   edge.append([unique_values[source_value], unique_values[target_value]])
          # print([unique_values[source_value], unique_values[target_value]])


    for i in entity_graph['context_level_links']:
        source_value = i[0].text
        target_value = i[1].text
        if source_value in unique_values and target_value in unique_values:
          if [unique_values[source_value], unique_values[target_value]] not in  edge:
            edge.append([unique_values[source_value], unique_values[target_value]])

    for i in entity_graph['paragraph_level_links']:
        source_value = i[0].text
        target_value = i[1].text
        if source_value in unique_values and target_value in unique_values:
          if [unique_values[source_value], unique_values[target_value]] not in  edge:
            edge.append([unique_values[source_value], unique_values[target_value]])

    # print(len(edge))
    return  edge,nodes,unique_values

# def encode_word(word):
#     token_ids = tz.encode(
#         word,
#         padding=True,
#         truncation=True,
#         max_length=512,
#         return_tensors="pt"
#     )
#     return token_ids

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv, GATv2Conv
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
from torch_geometric.data import Data  # Add this import
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import train_test_split


In [ ]:
import torch
from torch_geometric.nn import GATv2Conv
import torch.nn.functional as F
from transformers import BartForConditionalGeneration, BartTokenizer

# Define the GAT model
class GAT(torch.nn.Module):
    """Graph Attention Network"""
    def __init__(self, dim_in, dim_h, dim_out, heads=8):
        super().__init__()
        self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
        self.gat2 = GATv2Conv(dim_h * heads, dim_out, heads=1)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)

    def forward(self, x, edge_index):

        h = F.dropout(x, p=0.6, training=self.training)
        h = self.gat1(x, edge_index)
        h = F.elu(h)
        h = F.dropout(h, p=0.6, training=self.training)
        h = self.gat2(h, edge_index)
        return h, F.log_softmax(h, dim=1)



# Define the GAT-BART Decoder
class GAT_BART_Decoder(torch.nn.Module):
    def __init__(self, bart_model, bart_tokenizer):
        super(GAT_BART_Decoder, self).__init__()

        self.bart_model = bart_model
        self.bart_tokenizer = bart_tokenizer

    def forward(self, encoder_output , context, answer):
        encoder_output = encoder_output.mean(dim=1)

        input_ids = torch.cat((context[0], answer[0], encoder_output.detach()), dim=0).long()
        input_ids = input_ids.unsqueeze(0)

        output = self.bart_model.generate(input_ids)

        return output

# Initialize the GAT encoder and the GAT_BART_Decoder



In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch_geometric.data import Data
import numpy as np

# Define your custom dataset class
class CustomDataset(Dataset):
    def __init__(self,):
        self.data = Dataload()
        self.index=0

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        self.index+=1
        paragraph_data = self.data[idx]['paragraphs']

        # Build entity graph and generate nodes and edges
        entity_graph, N, entities = build_entity_graph(paragraph_data)
        # print("first")
        edge, nodes, unique_nodes = generate_graph(entities, entity_graph)
        if self.index%100==0:
         print("second",self.index )
        edge_index = torch.tensor(edge, dtype=torch.long).t().contiguous()
        x = torch.tensor(nodes, dtype=torch.float)

        # Extract context, answer, and question
        context = self.data[idx]['paragraphs']['p1']['context']+self.data[idx]['paragraphs']['p1']['context']
        answer = self.data[idx]['answer']
        question = self.data[idx]['question']

        # Create a Data object with nodes and edge information
        data = Data(x=x, edge_index=edge_index)

        return data, context, answer, question, unique_nodes




In [ ]:
import torch
import torch.nn as nn
from transformers import BartForConditionalGeneration

class CustomFineTuneModel(nn.Module):
    def __init__(self, gat_encoder, bart_decoder):
        super(CustomFineTuneModel, self).__init__()
        self.gat_encoder = gat_encoder
        self.bart_decoder = bart_decoder

    def forward(self, batch_data, context, answer, batch_questions):
        # Encode input using the GAT encoder
        x, edge_index = batch_data.x, batch_data.edge_index
        encoder_output, _ = self.gat_encoder(x, edge_index)
        output=self.bart_decoder(encoder_output, context, answer)


        return output


In [ ]:

def compute_loss(output, target):
  criteria = nn.CrossEntropyLoss()

  if len(output[0]) > len(target[0]):
    # Calculate the amount of padding needed
    padding_length = len(output[0]) - len(target[0])

    # Pad the tensor with zeros
    tar = F.pad(target[0], (0, padding_length), value=0)
    pred = output[0]


  if len(output[0]) < len(target[0]):
    # Calculate the amount of padding needed
    padding_length = len(target[0]) - len(output[0])

    # Pad the tensor with zeros
    pred = F.pad(output[0], (0, padding_length), value=0)
    tar =  target[0]
  loss = criteria(tar.float(), pred.float())
  torch.set_grad_enabled(True)
  loss.requires_grad = True


  return loss

import torch.nn.functional as F

def compute_accuracy(predictions, targets):
    # Ensure both sequences have the same length
    if len(predictions[0]) != len(targets[0]):
        # Calculate the amount of padding needed
        padding_length = abs(len(predictions[0]) - len(targets[0]))

        # Pad the tensor with zeros if necessary
        if len(predictions[0]) > len(targets[0]):
            tar = F.pad(targets[0], (0, padding_length), value=0)
            pred = predictions[0]
        else:
            pred = F.pad(predictions[0], (0, padding_length), value=0)
            tar = targets[0]
    else:
        pred = predictions[0]
        tar = targets[0]

    # Count the number of correct matches
    correct_matches = sum(1 for p, t in zip(pred, tar) if p == t)

    # Calculate accuracy as the ratio of correct matches to the total number of elements
    accuracy = correct_matches / len(pred)

    return accuracy


In [ ]:
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
dim_in = 1
dim_h = 64    # Set the hidden dimension
dim_out = 32  # Set the output dimension
heads = 8     # Set the number of attention heads
gat_encoder = GAT(dim_in, dim_h, dim_out, heads=heads)
gat_bart_decoder = GAT_BART_Decoder( bart_model, bart_tokenizer)

In [ ]:
CustomFineTuneModel = CustomFineTuneModel(gat_encoder, gat_bart_decoder)


In [ ]:
import torch
import time

start_time = time.time()




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define fine-tuning hyperparameters and optimizer (move optimizer definition here)
learning_rate = 0.001
batch_size = 16
num_epochs = 10

# Instantiate the optimizer with the model's parameters
optimizer = torch.optim.Adam(CustomFineTuneModel.parameters(), lr=learning_rate)
CustomFineTuneModel = CustomFineTuneModel.to(device)

# Initialize variables to keep track of the best model and lowest loss
best_loss = float('inf')
best_model_state_dict = None

# Prepare fine-tuning data
def fr(d):
    for i in range(0, len(d)):
        yield d[i]
tst = CustomDataset()
print(len(tst))
k = list(fr(tst))
# Split the data into train, validation, and test sets (60%, 20%, 20%)
train_data, val_data = train_test_split(k, test_size=0.2, random_state=42)
train_data, test_data = train_test_split(train_data, test_size=0.25, random_state=42)  # 20% for test, 60% for train
print("train_data",len(train_data))
print("test_size",len(test_data))
print("val_data",len(val_data))

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
100
second 100
train_data 60
test_size 20
val_data 20
Elapsed time: 95.74792265892029 seconds


In [ ]:
import pickle

# Save train_data to a file
with open('/content/drive/MyDrive/preproses_data_hotputQa/train_data.pkl', 'wb') as file:
    pickle.dump(train_data, file)
import pickle
with open('/content/drive/MyDrive/preproses_data_hotputQa/test_data.pkl', 'wb') as file:
    pickle.dump(test_data, file)
import pickle
with open('/content/drive/MyDrive/preproses_data_hotputQa/val_data.pkl', 'wb') as file:
    pickle.dump(val_data, file)
import pickle


In [ ]:
# Fine-tune the model
start_time = time.time()

for epoch in range(num_epochs):
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0

    # Training phase
    CustomFineTuneModel.train()
    for i in range(len(train_data)):
        if i%100==0:
         print("in train:",i)
        # batch_data, batch_contexts, batch_answers, batch_questions, batch_unique_nodes = train_data[i]
        # output = CustomFineTuneModel(batch_data, batch_contexts, batch_answers, batch_questions)
        batch_data, batch_contexts, batch_answers, batch_questions, batch_unique_nodes = train_data[i]
        batch_contexts =bart_tokenizer.encode(
            batch_contexts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        batch_answers = bart_tokenizer.encode(
            batch_answers,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
         )
        batch_questions = bart_tokenizer.encode(
            batch_questions,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        batch_data = batch_data.to(device)
        batch_contexts = batch_contexts.to(device)
        batch_answers = batch_answers.to(device)
        encode_question = batch_questions.to(device)

        output = CustomFineTuneModel(batch_data, batch_contexts, batch_answers, batch_questions)

        # Compute loss based on your task-specific criterion
        # # p=bart_tokenizer.decode(output)
        # tokenized_output = output.tolist()
        # print(tokenized_output[0],type(tokenized_output[0]))
        # # Decode the tokenized output
        # decoded_output = bart_tokenizer.decode(tokenized_output[0])
        # # z=art_tokenizer.decode(encode_question)
        # print(decoded_output)
        # # print(z)
        # print(type(output))
        loss = compute_loss(output, encode_question)
        total_loss += loss.item()

        # Define your target sequence here (adjust as needed)

        # Compute accuracy
        accuracy = compute_accuracy(output, encode_question)
        total_accuracy += accuracy

        num_batches += 1

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate average loss and accuracy for the epoch
    average_loss = total_loss / num_batches
    average_accuracy = total_accuracy / num_batches

    # Print training loss and accuracy for the epoch
    print(f'Training - Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f} - Accuracy: {average_accuracy:.4f}')

    # Validation phase
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0

    CustomFineTuneModel.eval()  # Set model to evaluation mode
    with torch.no_grad():
        for i in range(len(val_data)):
            if i%100==0:
             print("in val:",i%100==0)
            # batch_data, batch_contexts, batch_answers, batch_questions, batch_unique_nodes = val_data[i]
            # output = CustomFineTuneModel(batch_data, batch_contexts, batch_answers, batch_questions)
            batch_data, batch_contexts, batch_answers, batch_questions, batch_unique_nodes = train_data[i]
            batch_contexts =bart_tokenizer.encode(
                batch_contexts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            )
            batch_answers = bart_tokenizer.encode(
                batch_answers,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            )
            batch_questions = bart_tokenizer.encode(
                batch_questions,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            )
            batch_data = batch_data.to(device)
            batch_contexts = batch_contexts.to(device)
            batch_answers = batch_answers.to(device)
            encode_question = batch_questions.to(device)

            output = CustomFineTuneModel(batch_data, batch_contexts, batch_answers, batch_questions)


            loss = compute_loss(output, encode_question)
            total_loss += loss.item()


            # Compute accuracy
            accuracy = compute_accuracy(output, encode_question)
            total_accuracy += accuracy

            num_batches += 1

    # Calculate average loss and accuracy for the validation epoch
    average_loss = total_loss / num_batches
    average_accuracy = total_accuracy / num_batches

    # Print validation loss and accuracy for the epoch
    print(f'Validation - Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f} - Accuracy: {average_accuracy:.4f}')

    # Check if the current validation loss is lower than the best loss encountered so far
    if average_loss < best_loss:
        best_loss = average_loss
        best_model_state_dict = CustomFineTuneModel.state_dict()

# Save the best fine-tuned model
if best_model_state_dict is not None:
    torch.save(best_model_state_dict, 'best_fine_tuned_model.pth')

# Inference with the best fine-tuned model
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

in train: 0
Training - Epoch [1/10] - Loss: 4683429593.6000 - Accuracy: 0.0064
in val: True
Validation - Epoch [1/10] - Loss: 4729296499.2000 - Accuracy: 0.0023
in train: 0
Training - Epoch [2/10] - Loss: 4858240084.2667 - Accuracy: 0.0071
in val: True
Validation - Epoch [2/10] - Loss: 4729296499.2000 - Accuracy: 0.0023
in train: 0
Training - Epoch [3/10] - Loss: 4409123707.7333 - Accuracy: 0.0092
in val: True
Validation - Epoch [3/10] - Loss: 4729296499.2000 - Accuracy: 0.0023
in train: 0
Training - Epoch [4/10] - Loss: 4451081813.3333 - Accuracy: 0.0086
in val: True
Validation - Epoch [4/10] - Loss: 4729296499.2000 - Accuracy: 0.0023
in train: 0
Training - Epoch [5/10] - Loss: 4519149000.5333 - Accuracy: 0.0058
in val: True
Validation - Epoch [5/10] - Loss: 4729296499.2000 - Accuracy: 0.0023
in train: 0
Training - Epoch [6/10] - Loss: 4560083771.7333 - Accuracy: 0.0063
in val: True
Validation - Epoch [6/10] - Loss: 4729296499.2000 - Accuracy: 0.0023
in train: 0
Training - Epoch [7/10